# Carrier Pay Prediction (Super Dispatch-style)

This notebook builds an end-to-end ML pipeline to predict `Carrier_Pay` from shipment quote data using scikit-learn Pipelines and ColumnTransformer. It follows best practices: data cleaning, preprocessing, model comparison with cross-validation (RMSE, R²), hyperparameter tuning, and model persistence.

Expected columns:

- Quote_Id (will be dropped)
- Customer
- Vehicle_Types
- Mode(no HH)
- Total_Vehicles
- Origin_City
- Destination_City
- Origin_State
- Destination_State
- Origin_Zip
- Destination_Zip
- Miles
- Tariff
- GP
- Carrier_Pay (target)

If your CSV isn't present at `DATA_CSV_PATH`, this notebook auto-loads a small sample so it still runs end-to-end. Adjust `DATA_CSV_PATH` to your file before training on the full dataset.

In [13]:
# Imports & setup
import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_validate, KFold, train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

SEED = 42
np.random.seed(SEED)

# XGBoost optional
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

# OneHotEncoder compatibility for different sklearn versions
def make_ohe():
    try:
        return OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown='ignore', sparse=False)

print(f'XGBoost available: {HAS_XGB}')

XGBoost available: True


In [14]:
# Configure data path (update this if needed)
DATA_CSV_PATH = Path('data/quote_report_feb_dispatched.xlsx')  # Change to your file path if different

# If your CSV isn't available, we will auto-load a tiny sample so the notebook runs end-to-end.
SAMPLE_ROWS = [
    {
        'Quote_Id': 248092, 'Customer': 'Residential - Sales Jason Mishko', 'Vehicle_Types': 'Sedan Midsize(op)', 'Mode(no HH)': 'Open',
        'Total_Vehicles': 1, 'Origin_City': 'Palm Coast', 'Destination_City': 'Springfield', 'Origin_State': 'Florida', 'Destination_State': 'Missouri',
        'Origin_Zip': '32137', 'Destination_Zip': '65806', 'Miles': 1068, 'Tariff': 850, 'Carrier_Pay': 700, 'GP': 150
    },
    {
        'Quote_Id': 249030, 'Customer': 'Advantage Car Rental', 'Vehicle_Types': 'SUV Mid-size(op)', 'Mode(no HH)': 'Open',
        'Total_Vehicles': 1, 'Origin_City': 'Pearl', 'Destination_City': 'Orlando', 'Origin_State': 'Mississippi', 'Destination_State': 'Florida',
        'Origin_Zip': '39208', 'Destination_Zip': '32812', 'Miles': 691, 'Tariff': 1000, 'Carrier_Pay': 600, 'GP': 400
    },
    {
        'Quote_Id': 249289, 'Customer': 'Enterprise Fleet Management', 'Vehicle_Types': 'SUV(op)', 'Mode(no HH)': 'Open',
        'Total_Vehicles': 1, 'Origin_City': 'Schenectady', 'Destination_City': 'Cologne', 'Origin_State': 'New York', 'Destination_State': 'New Jersey',
        'Origin_Zip': '12304', 'Destination_Zip': '08215', 'Miles': 1, 'Tariff': 500, 'Carrier_Pay': 475, 'GP': 25
    },
]

if DATA_CSV_PATH.exists():
    df_raw = pd.read_excel(DATA_CSV_PATH,header=1)
    print(f'Loaded CSV: {DATA_CSV_PATH.resolve()} with shape {df_raw.shape}')
else:
    print(f'CSV not found at {DATA_CSV_PATH.resolve()}. Using an in-notebook sample to demonstrate the full pipeline.')
    df_raw = pd.DataFrame(SAMPLE_ROWS)
    print(df_raw.head())

Loaded CSV: C:\Users\omp\CascadeProjects\rw_ml\data\quote_report_feb_dispatched.xlsx with shape (1201, 18)


In [15]:
df.columns

Index(['Quote  Id', 'Customer', 'Vehicle Types', 'Mode(no HH)',
       'Total Vehicles', 'Origin City', 'Destination City', 'Origin State',
       'Destination State', 'Origin Zip', 'Destination Zip', 'Miles', 'Tariff',
       'Carrier Pay', 'GP', 'Tariff.1', 'Carrier Pay.1', 'GP.1'],
      dtype='object')

In [ ]:
df.rename(columns={
    'Quote  Id': 'quote_id',
    'Customer': 'customer',
    'Vehicle_Types': 'vehicle_types',
    'Mode(no HH)': 'mode',
    'Total_Vehicles': 'total_vehicles',
    'Origin_City': 'origin_city',
    'Destination_City': 'destination_city',
    'Origin_State': 'origin_state',
    'Destination_State': 'destination_state',
    'Origin_Zip': 'origin_zip',
    'Destination_Zip': 'destination_zip',
    'Miles': 'miles',
    'Tariff': 'tariff',
    'Carrier_Pay': 'carrier_pay',
    'GP': 'gp',
    'Tariff.1': 'tariff_api',
    'Carrier Pay.1': 'carrier_pay_api',
    'GP.1': 'gp_api'
    }, inplace=True)

In [17]:
# Data cleaning utilities
expected_categorical = ['customer','vehicle_types','mode','origin_city','destination_city','origin_state','destination_state']
expected_numeric = ['miles','tariff','gp','total_vehicles']
target_col = 'carrier_pay'

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Drop exact duplicates
    df = df.drop_duplicates()

    # Remove irrelevant identifier if present
    if 'quote_Id' in df.columns:
        df = df.drop(columns=['quote_Id'])

    # Ensure ZIPs are strings if present
    for col in ['origin_zip','destination_zip']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    # Convert numerics
    for col in ['miles','tariff','gp','total_vehicles', target_col]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Normalize categorical: strip whitespace
    for col in [c for c in expected_categorical if c in df.columns]:
        df[col] = df[col].astype(str).str.strip()

    # Remove invalid/zero/NaN mileage rows
    before = len(df)
    if 'miles' in df.columns:
        df = df[df['miles'].notna()]
        df = df[df['miles'] > 0]  # drop zero or negative miles
    removed = before - len(df)
    if removed:
        print(f'Removed {removed} rows with invalid miles (<=0 or NaN).')

    # Remove rows with missing target if present
    if target_col in df.columns:
        before = len(df)
        df = df[df[target_col].notna()]
        removed_t = before - len(df)
        if removed_t:
            print(f'Removed {removed_t} rows with missing {target_col}.')

    return df

df = clean_data(df_raw)
print('Cleaned shape:', df.shape)
display(df.head())

# Validate required columns exist
required_cols = expected_categorical + expected_numeric + [target_col]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f'Missing required columns: {missing}')

Cleaned shape: (1201, 18)


,Quote Id,Customer,Vehicle Types,Mode(no HH),Total Vehicles,Origin City,Destination City,Origin State,Destination State,Origin Zip,Destination Zip,Miles,Tariff,Carrier Pay,GP,Tariff.1,Carrier Pay.1,GP.1
0,248092,Residential - Sales Jason Mishko,Sedan Midsize(op),Open,1,Palm Coast,Springfield,Florida,Missouri,32137,65806,1068,850.0,700.0,150.0,1000.00,700,300.00
1,249030,Advantage Car Rental,SUV Mid-size(op),Open,1,Pearl,Orlando,Mississippi,Florida,39208,32812,691,1000.0,600.0,400.0,642.86,450,192.86
2,249289,Enterprise Fleet Management,SUV(op),Open,1,Schenectady,Cologne,New York,New Jersey,12304,8215,0,500.0,475.0,25.0,500.00,350,150.00
3,249582,Enterprise Fleet Management,Sedan Midsize(op),Open,1,Maitland,Rogers Park,Florida,Illinois,32751,60626,1145,1350.0,750.0,600.0,928.57,650,278.57
4,249618,Premier Auto Finance,Sedan Midsize(inop),Open,1,Ocala,Hollywood,Florida,Florida,34479,33023,293,550.0,500.0,50.0,571.43,400,171.43


ValueError: Missing required columns: ['customer', 'vehicle_types', 'mode', 'origin_city', 'destination_city', 'origin_state', 'destination_state', 'miles', 'tariff', 'gp', 'total_vehicles', 'carrier_pay']

In [ ]:
# Feature engineering via FunctionTransformer
def add_features(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    # Per-mile features (Miles>0 enforced by cleaning)
    if 'Miles' in X.columns and 'Tariff' in X.columns:
        denom = X['Miles'].replace(0, np.nan)
        X['tariff_per_mile'] = X['Tariff'] / denom
    if 'Miles' in X.columns and 'GP' in X.columns:
        denom = X['Miles'].replace(0, np.nan)
        X['gp_per_mile'] = X['GP'] / denom
    # Origin/Destination same-state flag
    if 'Origin_State' in X.columns and 'Destination_State' in X.columns:
        X['is_same_state'] = (
            X['Origin_State'].astype(str).str.strip().str.lower()
            == X['Destination_State'].astype(str).str.strip().str.lower()
        ).astype(int)
    else:
        X['is_same_state'] = 0
    return X

feature_adder = FunctionTransformer(add_features, validate=False)

categorical_cols = expected_categorical
numeric_base = expected_numeric
derived_numeric = ['tariff_per_mile','gp_per_mile','is_same_state']
numeric_all = numeric_base + derived_numeric

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', make_ohe()),
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_all),
        ('cat', categorical_transformer, categorical_cols),
    ],
    remainder='drop'
)

X = df[categorical_cols + numeric_base].copy()
y = df[target_col].values
print('X shape:', X.shape, '| y shape:', y.shape)
display(X.head())

In [ ]:
# Build model pipelines
def build_pipelines():
    models = {}
    models['linreg'] = Pipeline(steps=[
        ('features', feature_adder),
        ('preprocess', preprocess),
        ('model', LinearRegression()),
    ])

    models['rf'] = Pipeline(steps=[
        ('features', feature_adder),
        ('preprocess', preprocess),
        ('model', RandomForestRegressor(
            n_estimators=400,
            random_state=SEED,
            n_jobs=-1,
        )),
    ])

    models['gbr'] = Pipeline(steps=[
        ('features', feature_adder),
        ('preprocess', preprocess),
        ('model', GradientBoostingRegressor(random_state=SEED)),
    ])

    if HAS_XGB:
        models['xgb'] = Pipeline(steps=[
            ('features', feature_adder),
            ('preprocess', preprocess),
            ('model', XGBRegressor(
                n_estimators=600,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.9,
                colsample_bytree=0.8,
                reg_lambda=1.0,
                objective='reg:squarederror',
                n_jobs=-1,
                random_state=SEED,
                tree_method='hist',
                verbosity=0
            )),
        ])
    return models

models = build_pipelines()
print('Models:', list(models.keys()))

In [ ]:
# Evaluate base models with cross-validation
def evaluate_models(models, X, y):
    n_samples = len(y)
    if n_samples < 2:
        raise ValueError('Not enough rows to evaluate models.')
    cv_splits = min(5, n_samples)
    if cv_splits < 2:
        # Fallback simple holdout
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=SEED)
        rows = []
        for name, pipe in models.items():
            pipe.fit(X_train, y_train)
            pred = pipe.predict(X_test)
            rmse = mean_squared_error(y_test, pred, squared=False)
            r2 = r2_score(y_test, pred)
            rows.append({'model': name, 'rmse_mean': rmse, 'rmse_std': np.nan, 'r2_mean': r2, 'r2_std': np.nan, 'fit_time_s': np.nan})
        return pd.DataFrame(rows).sort_values('rmse_mean')
    else:
        cv = KFold(n_splits=cv_splits, shuffle=True, random_state=SEED)
        rows = []
        for name, pipe in models.items():
            res = cross_validate(
                pipe, X, y, cv=cv, n_jobs=-1,
                scoring={'rmse': 'neg_root_mean_squared_error', 'r2': 'r2'},
                return_train_score=False, error_score='raise'
            )
            rows.append({
                'model': name,
                'rmse_mean': -res['test_rmse'].mean(),
                'rmse_std': res['test_rmse'].std(),
                'r2_mean': res['test_r2'].mean(),
                'r2_std': res['test_r2'].std(),
                'fit_time_s': res['fit_time'].mean(),
            })
        return pd.DataFrame(rows).sort_values('rmse_mean')

results_df = evaluate_models(models, X, y)
display(results_df)

In [ ]:
# Hyperparameter tuning (optimize RMSE) for the best-performing model
def get_search_space(model_name):
    if model_name == 'rf':
        return {
            'model__n_estimators': [200, 400, 600, 800],
            'model__max_depth': [None, 10, 15, 20],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': ['sqrt', 'log2', 0.5],
        }
    if model_name == 'gbr':
        return {
            'model__n_estimators': [200, 400, 600],
            'model__learning_rate': [0.05, 0.1, 0.2],
            'model__max_depth': [2, 3, 4],
            'model__subsample': [0.6, 0.8, 1.0],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': ['sqrt', 'log2', None],
        }
    if model_name == 'xgb' and HAS_XGB:
        return {
            'model__n_estimators': [400, 600, 800],
            'model__learning_rate': [0.03, 0.05, 0.1],
            'model__max_depth': [4, 6, 8],
            'model__subsample': [0.7, 0.9, 1.0],
            'model__colsample_bytree': [0.6, 0.8, 1.0],
            'model__reg_lambda': [0.0, 1.0, 3.0],
            'model__reg_alpha': [0.0, 0.1, 0.5],
            'model__min_child_weight': [1, 3, 5],
        }
    return None

best_base = results_df.iloc[0]['model']
print('Best base model:', best_base)
param_space = get_search_space(best_base)

if param_space is None:
    print('No hyperparameters to tune or unsupported model. Proceeding with base model.')
    best_pipeline = models[best_base]
    best_pipeline.fit(X, y)
    best_cv_score = results_df.iloc[0]['rmse_mean']
    tuned = False
else:
    # RandomizedSearchCV for efficiency
    cv_splits = min(5, len(y))
    cv = KFold(n_splits=cv_splits, shuffle=True, random_state=SEED) if cv_splits >= 2 else 3
    search = RandomizedSearchCV(
        estimator=models[best_base],
        param_distributions=param_space,
        n_iter=25,
        scoring='neg_root_mean_squared_error',
        cv=cv,
        n_jobs=-1,
        random_state=SEED,
        verbose=1,
        refit=True,
        error_score='raise'
    )
    search.fit(X, y)
    best_pipeline = search.best_estimator_
    best_cv_score = -search.best_score_
    tuned = True

print(f'Tuned: {tuned}')
if tuned:
    print('Best hyperparameters:')
    print(json.dumps(search.best_params_, indent=2))
print(f'Best CV RMSE: {best_cv_score:.4f}')

In [ ]:
# Evaluate tuned/best pipeline with cross-validation for both RMSE and R^2
cv_splits = min(5, len(y))
if cv_splits >= 2:
    cv = KFold(n_splits=cv_splits, shuffle=True, random_state=SEED)
    res = cross_validate(
        best_pipeline, X, y, cv=cv, n_jobs=-1,
        scoring={'rmse': 'neg_root_mean_squared_error', 'r2': 'r2'},
        return_train_score=False
    )
    final_rmse = -res['test_rmse'].mean()
    final_r2 = res['test_r2'].mean()
else:
    # Fallback simple holdout
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.33, random_state=SEED)
    best_pipeline.fit(X_tr, y_tr)
    preds = best_pipeline.predict(X_te)
    final_rmse = mean_squared_error(y_te, preds, squared=False)
    final_r2 = r2_score(y_te, preds)

print(f'Final CV/holdout RMSE: {final_rmse:.4f} | R^2: {final_r2:.4f}')

In [ ]:
# Fit on full data and save the final model
best_pipeline.fit(X, y)
models_dir = Path('models')
models_dir.mkdir(parents=True, exist_ok=True)
model_path = models_dir / 'carrier_pay_model.joblib'
joblib.dump(best_pipeline, model_path)

metrics = {
    'best_base_model': str(results_df.iloc[0]['model']),
    'tuned': bool(tuned),
    'best_cv_rmse': float(best_cv_score),
    'final_rmse': float(final_rmse),
    'final_r2': float(final_r2)
}
with open(models_dir / 'carrier_pay_model_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'Model saved to: {model_path.resolve()}')
print('Metrics saved to:', (models_dir / 'carrier_pay_model_metrics.json').resolve())

In [ ]:
# Quick sanity check: predict on a few rows
sample_preds = best_pipeline.predict(X.head(5))
pd.DataFrame({
    'Carrier_Pay_actual': y[:5],
    'Carrier_Pay_pred': sample_preds
})